In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import re
from tqdm.auto import tqdm
import os
import sys
import torch
import csv
import numpy as np
import random
import time
import datetime
import pprint

In [2]:
import tensorflow as tf

In [3]:
from pathlib import Path
# from preprocess_c import *

In [4]:

from pathlib import Path
train_path  = '../data/protechn_corpus_eval/train'
test_path = '../data/protechn_corpus_eval/test'
dev_path = '../data/protechn_corpus_eval/dev'


In [5]:
# def make_dset(path):
#     path_ = Path(path)
#     a = make_dataset(path_)
#     df_1 = pd.DataFrame(columns=['id','full_sent','start_sent','end_sent','start_prop','end_prop','prop','??','???'])
#     for dm in a:
#         df_t = pd.DataFrame(dm,columns =['id','full_sent','start_sent','end_sent','start_prop','end_prop','prop','??','???'] )
#         df_1 = df_1.append(df_t,ignore_index= True)
#     return df_1.iloc[:,:-2]

In [6]:
#df_train = make_dset(train_path)
df_train= pd.read_csv('ADD5_Context5.csv')
df_test = pd.read_csv('tuan_test_data.csv')

In [7]:
# df_test[df_test['full_sent'].str.len() > 0]

In [8]:
lonme = [0 if i == 'O' else 1 for i in df_train.prop.values ]
df_train['binary'] = lonme


In [9]:
test_bi = [0 if i == 'O' else 1 for i in df_test.prop.values ]
df_test['binary'] = test_bi


In [10]:
mapping = {'Loaded_Language':1,'Name_Calling,Labeling':2,'Repetition':3,
           'Exaggeration,Minimisation':4,'Doubt':5,'Appeal_to_fear-prejudice':6,'Flag-Waving':7,'Causal_Oversimplification':8,
           'Slogans':9,'Appeal_to_Authority':10,'Black-and-White_Fallacy':11,'Thought-terminating_Cliches':12,'Whataboutism':13,
           'Reductio_ad_hitlerum':14,'Red_Herring':15,'Bandwagon':16,'Obfuscation,Intentional_Vagueness,Confusion':17,'Straw_Men':18,'O':0}
#df_train = df_train[df_train.binary !=0]

df_train['prop_1'] = df_train.prop.apply(lambda x: mapping[x])
df_test['prop_1'] = df_test.prop.apply(lambda x: mapping[x])
# df_train

In [11]:
df_test_nm = df_test[df_test['prop'] != "O"]


In [12]:
# import glob
# import os
# train_direct = glob.glob('data/data_propoganda/data/protechn_corpus_eval/train/*.txt')
# articles = []
# def read_articles():
#   for filename in train_direct:
#     myfile = open(filename)
#     article = myfile.read()
#     articles.append(article)
#     myfile.close()
#   article_ids = []
#   for filename in train_direct:
#     article_ids.append(filename[60:-4])
  
#   return articles, article_ids
# articles,art_ids = read_articles()
# id2art ={i:a for a,i in zip(articles,art_ids)}

In [13]:
# def get_context(article, span, mode='sentence'):
#   article = id2art[article]
#   def get_num_words(sentence):
#     return len(sentence.split(' '))
#   if mode == "title":
#     return article.split('\n')[0]
#   if mode == "sentence":
#     WORD_LEN_LIMIT = 120
#     li = span[0]
#     ri = span[1]
#     span_text = article[li: ri]
#     num_words = get_num_words(span_text)
#     if num_words >= WORD_LEN_LIMIT:
#       return span_text
#     remaining_len = WORD_LEN_LIMIT - num_words
#     lhs_words = remaining_len // 2
#     rhs_words = remaining_len - lhs_words
#     li -= 1
#     lcount = 0
#     while li >= 0 and article[li-1] != '\n' and lcount < lhs_words:
#       if article[li] == ' ':
#         lcount += 1
#       li -= 1
#     ri += 1
#     rcount = 0
#     while ri < len(article) and article[ri] != '\n' and rcount < rhs_words:
#       if article[ri] == ' ':
#         rcount += 1
#       ri += 1
#     return article[li+1: ri - 1] 

#   return ""
# spans_1 = [(i,k,j) for i,k,j in zip(df_train.id,df_train.start_sent,df_train.end_sent)]
# df_train['context'] = [get_context(i,(s,e)) for i,s,e in spans_1]

In [14]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import BertTokenizer

In [15]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [16]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.full_sent[index])
        title = " ".join(title.split())
        context = str(self.data.context[index])
        context = " ".join(context.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        tokenized_context = self.tokenizer.encode_plus(context,
                                            add_special_tokens=True,
                                            max_length=self.max_len,
                                            pad_to_max_length=True,
                                            return_attention_mask=True,truncation = True)

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        c_ids = tokenized_context['input_ids']
        c_mask = tokenized_context['attention_mask']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.prop_1[index], dtype=torch.long),
            'c_ids':torch.tensor(c_ids, dtype=torch.long),
            'c_mask': torch.tensor(c_mask,dtype= torch.long)
        } 
    
    def __len__(self):
        return self.len

In [17]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
# VALID_RARE_BATCH_SIZE = int(df_test_nm.__len__()/4)

EPOCHS = 20
LEARNING_RATE = 1e-05

In [18]:
train_size = 1
train_dataset=df_train.sample(frac=train_size,random_state=200)
# test_dataset=df_train.drop(train_dataset.index).reset_index(drop=True)
test_dataset = df_test.reset_index(drop=True)
test_nm_dataset = df_test_nm.reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df_train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))
print("TEST Non-majourity Dataset: {}".format(test_nm_dataset.shape))
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)
testing_nm_set = Triage(test_nm_dataset, tokenizer, MAX_LEN)

FULL Dataset: (42948, 10)
TRAIN Dataset: (42948, 10)
TEST Dataset: (4490, 10)
TEST Non-majourity Dataset: (1490, 10)


In [19]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0,
                'drop_last': False
                }


# test_nm_params = {'batch_size': VALID_RARE_BATCH_SIZE,
#                 'shuffle': False,
#                 'num_workers': 0,
#                 'drop_last': False
#                 }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
# testing_nm_loader = DataLoader(testing_nm_set, **test_nm_params)


In [20]:
from torch.nn import CrossEntropyLoss, MSELoss,BCEWithLogitsLoss

class ContextualBertForSequenceClassification(torch.nn.Module):
  
  def __init__(self, num_labels, ContextModel, SpanModel):
    super(ContextualBertForSequenceClassification, self).__init__()
    self.ContextModel = ContextModel
    self.SpanModel = SpanModel
    self.num_labels = num_labels

    # self.classifier = torch.nn.Linear(768*2, num_labels)
    # self.classifier1 = torch.nn.Linear(768, num_labels)
    self.classifier2 = torch.nn.Linear(768+128, num_labels)
    self.reduce_classifier = torch.nn.Linear(768, 128)
    self.dropout = torch.nn.Dropout(0.1)

  def forward(
      self,
      span_input_ids,
      span_attention_mask,
      context_input_ids,
      context_attention_mask,
      labels=None
  ):
    context_outputs = self.ContextModel(
        input_ids=context_input_ids,
        attention_mask=context_attention_mask
    )
    context_outputs = context_outputs[1] # pooler output
    span_outputs = self.SpanModel(
        input_ids=span_input_ids,
        attention_mask=span_attention_mask
    )
    span_outputs = span_outputs[1]

    context_outputs = self.reduce_classifier(context_outputs)
    pooled_output = torch.cat((span_outputs, context_outputs), axis=1)

    pooled_output = self.dropout(pooled_output)

    logits = self.classifier2(pooled_output)
    outputs = (logits,)
    if labels is not None:
      if self.num_labels == 1:
        loss_fct = MSELoss()
        loss = loss_fct(logits.view(-1), labels.view(-1))
      else:
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
      outputs = (loss,) + outputs

    return outputs

In [21]:
"""class DistillBERTBinary(torch.nn.Module):
    def __init__(self):
        super(DistillBERTBinary, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 1)
        self.softmax = torch.nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output =self.classifier(pooler)
        return output"""

'class DistillBERTBinary(torch.nn.Module):\n    def __init__(self):\n        super(DistillBERTBinary, self).__init__()\n        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")\n        self.pre_classifier = torch.nn.Linear(768, 768)\n        self.dropout = torch.nn.Dropout(0.3)\n        self.classifier = torch.nn.Linear(768, 1)\n        self.softmax = torch.nn.Sigmoid()\n\n    def forward(self, input_ids, attention_mask):\n        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)\n        hidden_state = output_1[0]\n        pooler = hidden_state[:, 0]\n        pooler = self.pre_classifier(pooler)\n        pooler = torch.nn.ReLU()(pooler)\n        pooler = self.dropout(pooler)\n        output =self.classifier(pooler)\n        return output'

In [22]:
"""# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 18)
        self.softmax = torch.nn.Softmax(19)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output"""

'# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. \n\nclass DistillBERTClass(torch.nn.Module):\n    def __init__(self):\n        super(DistillBERTClass, self).__init__()\n        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")\n        self.pre_classifier = torch.nn.Linear(768, 768)\n        self.dropout = torch.nn.Dropout(0.3)\n        self.classifier = torch.nn.Linear(768, 18)\n        self.softmax = torch.nn.Softmax(19)\n\n    def forward(self, input_ids, attention_mask):\n        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)\n        hidden_state = output_1[0]\n        pooler = hidden_state[:, 0]\n        pooler = self.pre_classifier(pooler)\n        pooler = torch.nn.ReLU()(pooler)\n        pooler = self.dropout(pooler)\n        output = self.classifier(pooler)\n        return output'

In [23]:

def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
def train(model, epochs=5):
  loss_values = []
  for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_loss = 0
    model.train()
    for step, batch in enumerate(training_loader):
      if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(training_loader), elapsed))
      b_input_ids = batch['ids'].to(device)
      b_labels = batch['targets'].to(device, dtype = torch.long)
      b_input_mask = batch['mask'].to(device)
      b_c_input_ids = batch['c_ids'].to(device)
      b_c_input_mask = batch['c_mask'].to(device)
      model.zero_grad()        
      outputs = model(b_input_ids, 
                      b_input_mask,
                      b_c_input_ids, 
                      b_c_input_mask, 
                      labels=b_labels)
      loss = outputs[0]
      total_loss += loss.item()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step() # TODO
      stats = '[%d/%d][%d/%d]\tLoss: %.4f' \
                % (epoch_i+1, epochs, step, len(training_loader), loss.item())
      print('\r' + stats, end="")
      sys.stdout.flush()
    avg_train_loss = total_loss / len(training_loader)            
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in testing_loader:
      # batch = tuple(t.to(device) for t in batch)
      b_input_ids = batch['ids'].to(device)
      b_labels = batch['targets'].to(device, dtype = torch.long)
      b_input_mask = batch['mask'].to(device)
      b_c_input_ids = batch['c_ids'].to(device)
      b_c_input_mask = batch['c_mask'].to(device)
      with torch.no_grad():        
        outputs = model(b_input_ids, 
                        b_input_mask,
                        b_c_input_ids, 
                        b_c_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      eval_accuracy += tmp_eval_accuracy
      nb_eval_steps += 1
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
  print("")
  print("Training complete!")
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [25]:
def get_model_predictions(model, dataloader):
  model.eval()
  predictions , true_labels = [], []
  nb_eval_steps = 0
  for batch in dataloader:
    b_input_ids = batch['ids'].to(device)
    b_labels = batch['targets'].to(device)
    b_input_mask = batch['mask'].to(device)
    b_c_input_ids = batch['c_ids'].to(device)
    b_c_input_mask = batch['c_mask'].to(device)
    with torch.no_grad():        
      logits = model(b_input_ids, 
                     b_input_mask,
                     b_c_input_ids, 
                     b_c_input_mask)
    logits = logits[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    pred_label = np.argmax(logits, axis=1) #[1 if a >0.5 else 0 for a in logits]
    predictions.extend(pred_label)
    true_labels.extend(label_ids)
  return predictions, true_labels

In [26]:
def get_dev_predictions(model):
  test_articles, _ = read_articles("dev-articles")
  test_spans, test_techniques = read_test_spans()

  test_articles = test_articles[1:]
  test_dataloader = get_data(test_articles, test_spans, test_techniques)
  pred, _ = get_model_predictions(model, test_dataloader)

  with open('predictions.txt', 'w') as fp:
    label_file = os.path.join(data_dir, "dev-task-TC-template.out")
    myfile = open(label_file)
    prev_index = -1
    tsvreader = csv.reader(myfile, delimiter="\t")
    for i, row in enumerate(tsvreader):
      fp.write(row[0] + '\t' + distinct_techniques[pred[i]] + '\t' + row[2] + '\t' + row[3] + '\n')

In [27]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
#from transformers import RobertaModel
from transformers import BertModel
#from transformers import RobertaForSequenceClassification
import time,sys

model_name = 'bert-base-cased'
context_model = BertModel.from_pretrained(model_name)
span_model = BertModel.from_pretrained(model_name)
model = ContextualBertForSequenceClassification(19, context_model, span_model)
model.cuda()

ContextualBertForSequenceClassification(
  (ContextModel): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), 

In [28]:


optimizer = AdamW(model.parameters(),lr = 3e-5,eps = 1e-8) # ler = 5e-5
epochs = 7

total_steps = len(training_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
train(model, epochs=epochs)


======== Epoch 1 / 7 ========
Training...


/m/home/home2/26/nguyenl21/unix/DSP/venv/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[1/7][99/1343]	Loss: 2.1379  Batch   100  of  1,343.    Elapsed: 0:01:40.
[1/7][199/1343]	Loss: 1.5695  Batch   200  of  1,343.    Elapsed: 0:03:22.
[1/7][299/1343]	Loss: 1.8469  Batch   300  of  1,343.    Elapsed: 0:05:05.
[1/7][399/1343]	Loss: 1.0334  Batch   400  of  1,343.    Elapsed: 0:06:48.
[1/7][499/1343]	Loss: 1.2125  Batch   500  of  1,343.    Elapsed: 0:08:30.
[1/7][599/1343]	Loss: 1.3641  Batch   600  of  1,343.    Elapsed: 0:10:12.
[1/7][699/1343]	Loss: 0.8691  Batch   700  of  1,343.    Elapsed: 0:11:55.
[1/7][799/1343]	Loss: 0.9900  Batch   800  of  1,343.    Elapsed: 0:13:37.
[1/7][899/1343]	Loss: 1.3946  Batch   900  of  1,343.    Elapsed: 0:15:19.
[1/7][999/1343]	Loss: 1.1283  Batch 1,000  of  1,343.    Elapsed: 0:17:01.
[1/7][1099/1343]	Loss: 1.0795  Batch 1,100  of  1,343.    Elapsed: 0:18:43.
[1/7][1199/1343]	Loss: 1.0621  Batch 1,200  of  1,343.    Elapsed: 0:20:26.
[1/7][1299/1343]	Loss: 0.7990  Batch 1,300  of  1,343.    Elapsed: 0:22:08.
[1/7][1342/1343]	Loss: 

In [29]:
pred,true = get_model_predictions(model, testing_loader)

In [30]:
sum([a==b for a,b in zip(pred,true)])/len(pred)

0.6879732739420935

In [31]:
from sklearn.metrics import classification_report

In [32]:
print(classification_report(true, pred))

              precision    recall  f1-score   support

           0       0.76      0.94      0.84      3000
           1       0.41      0.38      0.39       436
           2       0.37      0.21      0.27       209
           3       0.11      0.01      0.01       195
           4       0.24      0.09      0.13        99
           5       0.10      0.07      0.08        89
           6       0.32      0.12      0.18       130
           7       0.48      0.21      0.29        96
           8       0.06      0.03      0.04        33
           9       0.25      0.06      0.09        36
          10       0.00      0.00      0.00        56
          11       0.00      0.00      0.00        26
          12       0.00      0.00      0.00        22
          13       0.33      0.04      0.07        25
          14       0.25      0.09      0.13        11
          15       0.00      0.00      0.00        15
          16       0.00      0.00      0.00         4
          17       0.00    

/work/modules/Ubuntu/14.04/amd64/common/anaconda3/20190819-gpu/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
torch.save(model.state_dict(), 'ADD5_Context5_augmented.pth')

In [34]:
model.load_state_dict(torch.load('Context_only_augmented.pth'))
# model.eval()

<All keys matched successfully>

In [ ]:
pred_nm,true_nm = get_model_predictions(model, testing_nm_loader)
sum([a==b for a,b in zip(pred_nm,true_nm)])/len(pred_nm)

In [ ]:
# **testing_nm_loader

In [ ]:
print(classification_report(true_nm, pred_nm))

In [ ]:
df_test_nm.groupby(['prop']).count()

In [ ]:
from collections import Counter
Counter(torch.cat([
    x['targets'] for x in testing_nm_loader
]).numpy())

In [ ]:
pred_nm